In [1]:
s = input("Enter the input string: ")
size = 64 
n = len(s)
m = n * 8

msglen = format(m, '064b') 
ans = ""
res = []

for i in range(0, n, size):
    chunk = s[i:i+size]

    for j in range(len(chunk)):
        temp = format(ord(chunk[j]), '08b')
        ans += temp
    
    if i + size >= n:
        ans += '1'
        req = (448 - len(ans) % 512) % 512
        ans += '0' * req
        ans += msglen

    while len(ans) >= 512:
        block = ans[:512]
        ans = ans[512:]
        words = []
        for k in range(0, 512, 32):
            words.append(block[k:k+32])
        res.append(words)

# Print the final result
print(f"Number of 512-bit chunks: {len(res)}")
for idx, chunk in enumerate(res):
    print(f"Chunk {idx + 1}: {chunk}")


Number of 512-bit chunks: 1
Chunk 1: ['01100010011001001000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000010000']


In [9]:
import struct
import math

def left_rotate(x, c):
    return ((x << c) | (x >> (32 - c))) & 0xFFFFFFFF

def md5_manual(message):
    A, B, C, D = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476
    K = [int(abs(math.sin(i + 1)) * (2**32)) & 0xFFFFFFFF for i in range(64)]
    S = [
        [7, 12, 17, 22], [5, 9, 14, 20],
        [4, 11, 16, 23], [6, 10, 15, 21]
    ]
    original_len_bits = len(message) * 8
    message += b'\x80'
    message += b'\x00' * ((56 - (len(message) % 64)) % 64)
    message += struct.pack('<Q', original_len_bits)

    for i in range(0, len(message), 64):
        chunk = message[i:i+64]
        M = struct.unpack('<16I', chunk)
        a, b, c, d = A, B, C, D

        for j in range(64):
            if j < 16:
                f, g, shift = (b & c) | (~b & d), j, S[0][j % 4]
            elif j < 32:
                f, g, shift = (d & b) | (~d & c), (5 * j + 1) % 16, S[1][j % 4]
            elif j < 48:
                f, g, shift = b ^ c ^ d, (3 * j + 5) % 16, S[2][j % 4]
            else:
                f, g, shift = c ^ (b | ~d), (7 * j) % 16, S[3][j % 4]
            temp = (a + f + K[j] + M[g]) & 0xFFFFFFFF
            a, b, c, d = d, (b + left_rotate(temp, shift)) & 0xFFFFFFFF, b, c

        A, B, C, D = (A + a) & 0xFFFFFFFF, (B + b) & 0xFFFFFFFF, (C + c) & 0xFFFFFFFF, (D + d) & 0xFFFFFFFF

    return f"{A:08x}{B:08x}{C:08x}{D:08x}"

if __name__ == "__main__":
    message = b"This is a message sent by a computer user."
    print("MD5 Hash:", md5_manual(message))

MD5 Hash: e84725928f9bc866df127367eec8ce0c


In [8]:
import hashlib
# making a message
inputstring = "This is a message sent by a computer user."
# encoding the message using the library function
output = hashlib.md5(inputstring.encode())
# printing the hash function
print("Hash of the input string:")
print(output.hexdigest())

Hash of the input string:
922547e866c89b8f677312df0ccec8ee
